# カートポールのバランスを取るためのRLトレーニング

このノートブックは[AI for Beginners Curriculum](http://aka.ms/ai-beginners)の一部です。[公式PyTorchチュートリアル](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)や[このCartpole PyTorch実装](https://github.com/yc930401/Actor-Critic-pytorch)に触発されています。

この例では、強化学習（RL）を使用して、左右に動くカートの上でポールのバランスを取るモデルをトレーニングします。[OpenAI Gym](https://www.gymlibrary.ml/)環境を使用してポールのシミュレーションを行います。

> **Note**: このレッスンのコードはローカル環境（例: Visual Studio Code）で実行することができます。この場合、シミュレーションは新しいウィンドウで開きます。オンラインでコードを実行する場合、[こちら](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7)に記載されているように、コードにいくつかの調整が必要になる場合があります。

まず、Gymがインストールされていることを確認します:


In [ ]:
import sys
!{sys.executable} -m pip install gym

では、CartPole環境を作成し、その操作方法を見てみましょう。環境には以下の特性があります：

* **Action space** は、シミュレーションの各ステップで実行可能な行動の集合です  
* **Observation space** は、観測可能な情報の空間です  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

シミュレーションがどのように動作するか見てみましょう。以下のループは、`env.step` が終了フラグ `done` を返すまでシミュレーションを実行します。ここでは、`env.action_space.sample()` を使ってランダムにアクションを選択します。つまり、この実験はおそらく非常に早く失敗するでしょう（CartPole 環境は、CartPole の速度、位置、または角度が特定の制限を超えた場合に終了します）。

> シミュレーションは新しいウィンドウで開きます。このコードを何度か実行して、その挙動を確認してみてください。


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

観測には4つの数値が含まれていることに気づくでしょう。それらは以下の通りです：
- カートの位置
- カートの速度
- ポールの角度
- ポールの回転速度

`rew`は各ステップで受け取る報酬です。CartPole環境では、シミュレーションの各ステップごとに1ポイントの報酬が与えられます。目標は総報酬を最大化すること、つまりCartPoleが倒れずにバランスを保つ時間を延ばすことです。

強化学習では、私たちの目標は**ポリシー** $\pi$ を訓練することです。このポリシーは各状態 $s$ に対してどのアクション $a$ を取るべきかを教えてくれます。つまり、基本的には $a = \pi(s)$ となります。

確率的な解法を求める場合、ポリシーは各アクションに対する確率の集合を返すものと考えることができます。例えば、$\pi(a|s)$ は状態 $s$ においてアクション $a$ を取るべき確率を意味します。

## ポリシー勾配法

最も単純な強化学習アルゴリズムである**ポリシー勾配法**では、次のアクションを予測するためにニューラルネットワークを訓練します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

ネットワークを訓練するために、多くの実験を実行し、各実行後にネットワークを更新します。実験を実行し、結果（いわゆる**トレース**）を返す関数を定義しましょう - すべての状態、行動（およびその推奨確率）、報酬を含みます:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

未訓練のネットワークで1つのエピソードを実行し、総報酬（エピソードの長さ）が非常に低いことを確認できます。


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

ゲームの各ステップで総報酬のベクトルを計算し、その過程でいくつかの係数 $gamma$ を使用して初期の報酬を割引するというアイデアです。また、トレーニングに影響を与える重みとして使用するため、結果のベクトルを正規化します。


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

さあ、実際のトレーニングを始めましょう！300エピソードを実行し、各エピソードで以下の手順を行います：

1. 実験を実行し、トレースを収集します。
2. 実行したアクションと予測された確率の差（`gradients`）を計算します。この差が小さいほど、正しいアクションを選択した確信が高まります。
3. 割引報酬を計算し、それを`gradients`に掛け合わせます。これにより、報酬が高いステップが最終結果に低い報酬のステップよりも大きな影響を与えるようになります。
4. ニューラルネットワークの期待されるターゲットアクションは、実行中に予測された確率の一部と、計算された`gradients`の一部から取得されます。`alpha`パラメータを使用して、`gradients`と報酬がどの程度考慮されるかを決定します。これを強化学習アルゴリズムの*学習率*と呼びます。
5. 最後に、状態と期待されるアクションを基にネットワークをトレーニングし、このプロセスを繰り返します。


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

では、レンダリングを行いながらエピソードを実行して結果を確認しましょう。


In [ ]:
_ = run_episode(render=True)

うまくいけば、ポールがかなりうまくバランスを取れるようになったことがわかるでしょう！

## アクター・クリティックモデル

アクター・クリティックモデルは、ポリシーグラデーションのさらなる発展形であり、ポリシーと推定報酬の両方を学習するためのニューラルネットワークを構築します。このネットワークには2つの出力（または2つの別々のネットワークとして見ることもできます）があります：
* **アクター**は、ポリシーグラデーションモデルのように、状態の確率分布を提供して次に取るべきアクションを推奨します。
* **クリティック**は、それらのアクションから得られる報酬を推定します。与えられた状態で将来得られる総推定報酬を返します。

このようなモデルを定義してみましょう：


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

私たちは `discounted_rewards` と `run_episode` 関数を少し修正する必要があります。


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


これからメインのトレーニングループを実行します。適切な損失関数を計算し、ネットワークパラメータを更新することで、手動のネットワークトレーニングプロセスを使用します。


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## まとめ

このデモでは、2つの強化学習アルゴリズムを紹介しました：シンプルなポリシー勾配と、より洗練されたアクタークリティックです。これらのアルゴリズムは、状態、行動、報酬といった抽象的な概念を扱うため、非常に異なる環境にも適用可能であることが分かります。

強化学習では、最終的な報酬を観察するだけで問題を解決する最適な戦略を学ぶことができます。ラベル付きデータセットが不要であるため、シミュレーションを何度も繰り返してモデルを最適化することが可能です。しかし、強化学習にはまだ多くの課題が存在しており、この興味深いAI分野にさらに注力することで、それらについて学ぶことができるでしょう。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は責任を負いません。
